In [ ]:
#this notebook exports emission data to csv after loading emission data (downloaded from the CCI server) to your local server

#set up working environment
import emission.core.get_database as edb
import pandas as pd
import numpy as np
import re

#view all users in the 
all_users = pd.DataFrame(list(edb.get_uuid_db().find({},{"user_email":1, "uuid": 1, "_id": 0})))
all_users

In [ ]:
from uuid import UUID

In [ ]:
import emission.storage.timeseries.abstract_timeseries as esta
import emission.storage.decorations.analysis_timeseries_queries as esda
import emission.core.wrapper.entry as ecwe
import emission.storage.decorations.trip_queries as esdt
import emission.storage.timeseries.timequery as estt

In [ ]:
import arrow

#iterate through each token to create a table of emission data
frames = []
for test_user_id in all_users.uuid:
    ts = esta.TimeSeries.get_time_series(test_user_id)
    #get all cleaned trips for user
    entry_it = ts.find_entries(["analysis/cleaned_trip"], time_query=None)
    print("User %s had %d trips" % (test_user_id, len(list(entry_it))))
    #get all cleaned trip data for the user
    #ct_df = ts.get_data_df("analysis/cleaned_trip", time_query=None)
    #get all cleaned section data for the user - we use sections in order to get mode
    cs_df = ts.get_data_df("analysis/cleaned_section", time_query=None)
    frames.append(cs_df.iloc[:, ::-1])

sections = pd.concat(frames).reset_index().drop('index', axis=1).rename(columns={'user_id': 'uuid'})

sections

In [ ]:
#function to include token with uuid
def createMappings(file_name, current_users):
    tokens=pd.read_csv(file_name)
    tokens=tokens.drop(["Unnamed: 0"], axis=1)
    current_users["uuid_str"]=np.array([str(i) for i in current_users["uuid"]])
    joined_table=current_users.merge(tokens, left_on='uuid_str', right_on='uuid')
    joined_table=joined_table.drop(["uuid_str","uuid_y","user_email_x"], axis=1)
    joined_table=joined_table.rename(index=str, columns={"user_email_y":"Token","uuid_x":"uuid"})
    return joined_table

In [ ]:
#the csv was generated while downloading data from the original server
tokens = createMappings("servertokens_0717.csv", all_users)
tokens

In [ ]:
#join tokens to sections table
table_with_tokens = pd.merge(sections, tokens, how='left', on= 'uuid')
cols = table_with_tokens.columns.tolist()
cols= cols[-1:] + cols[:-1]
sections_with_tokens = table_with_tokens[cols]

In [ ]:
#separate start lat/lon
start_loc_lat= []
start_loc_lon= []

for i in (sections_with_tokens['start_loc']):
    start_loc_lon.append(re.findall(r"-?\d+\.\d+", str(i))[0])
    start_loc_lat.append(re.findall(r"-?\d+\.\d+", str(i))[1])
    
sections_with_tokens['start_loc_lat']= start_loc_lat
sections_with_tokens['start_loc_lon']= start_loc_lon

In [ ]:
#separate end lat/lon
end_loc_lat= []
end_loc_lon= []

for i in (sections_with_tokens['end_loc']):
    end_loc_lon.append(re.findall(r"-?\d+\.\d+", str(i))[0])
    end_loc_lat.append(re.findall(r"-?\d+\.\d+", str(i))[1])
    
sections_with_tokens['end_loc_lat']= end_loc_lat
sections_with_tokens['end_loc_lon']= end_loc_lon

In [ ]:
#add new lat/lon columns to data frame
sections_tokens = sections_with_tokens.drop(['start_loc', 'end_loc','_id'], axis=1)
cols = sections_tokens.columns.tolist()
cols= cols[0:12]+ cols[-4:-2] + cols[12:26] + cols[-2:]+ cols[26:-4]
sections_1 = sections_tokens[cols]
sections_1

In [ ]:
#convert duration from seconds into minutes
sections_1['duration']= round(sections_1['duration']/60, 2)

#convert distance into kilometers
sections_1['distance'] = sections_1['distance']/1000

In [ ]:
#replace sensed_mode with actual mode
mode = pd.read_csv("sensed_mode.csv")
df_with_mode = sections_1.merge(mode,on='sensed_mode')
df_with_mode

In [ ]:
#indicate if trip is made on a weekday
weekday = pd.read_csv("weekday.csv")
emission_table = df_with_mode.merge(weekday,on='start_local_dt_weekday')
emission_table

In [ ]:
#export to csv
emission_table.to_csv("emission_sections_0717.csv")